In [1]:
from Inverse import *
from q_method import *

In [3]:
N = 64
k = [20]
m = 64
maxq = 0.1
q_method = 'G'
noise_level = 0.0
gtol = 1e-10
maxiter = 20

In [4]:
sys.path.append('/data/liuziyang/Programs/pde_solver')
if isinstance(k,str):
    tmp_k = k.split(',')
    k = [float(eval(item)) for item in tmp_k]
k_len = len(k)
NET = torch.load('/data/liuziyang/Programs/pde_solver/Network/k40_P_4,64,uniform_G_0.1_NST_R200_12,32,4_1.pth')
q = q_gen(N, q_method, maxq)
Q = q.reshape(-1, )
Q0 = Q*0
matrix_A = gen_A(N)
Matrix_analysis(N)
f_data = np.zeros((k_len,m,(N+1)**2),dtype = np.complex128)
partial_data = np.zeros((k_len,m,4*N-4),dtype = np.complex128)
for j in range(k_len):
    f_data[j] = f_gen(N, k[j], m)
    partial_data[j] = pdata_gen(N, Q, k[j], f_data[j], matrix_A,noise_level)

f_data_torch = torch.stack([torch.from_numpy(f_data.reshape(k_len,m,(N+1),(N+1)).real).to(torch.float32).to(device),
                           torch.from_numpy(f_data.reshape(k_len,m,(N+1),(N+1)).imag).to(torch.float32).to(device)],2)
partial_data_torch = torch.stack([torch.from_numpy(partial_data.reshape(k_len,m,4*N-4).real).to(torch.float32).to(device),
                           torch.from_numpy(partial_data.reshape(k_len,m,4*N-4).imag).to(torch.float32).to(device)],2)
matrix_A_torch = torch.from_numpy(matrix_A).to(torch.float32).to(device)
matrix_A_torch = matrix_A_torch.unsqueeze(0).unsqueeze(1).repeat(m,2,1,1)
f_data = (f_data, f_data_torch)
partial_data = (partial_data,partial_data_torch)
matrix_A = (matrix_A,matrix_A_torch)

In [5]:
%timeit data_np = J(Q,N,partial_data[0][0],k[0],f_data[0][0],matrix_A[0])
%timeit data_torch = J_NET(Q,N,partial_data[1][0],k[0],f_data[1][0],matrix_A[1],NET,device,3)
print('MUMPS: {}, NET: {}'.format(data_np, data_torch))

value
value
value
value
value
value
value
value
1.31 s ± 211 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
value
74.1 ms ± 94.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


NameError: name 'data_np' is not defined

In [6]:
%timeit grad_np = J_prime(Q,N,partial_data[0][0],k[0],f_data[0][0],matrix_A[0])
%timeit grad_torch = J_prime_NET(Q,N,partial_data[1][0],k[0],f_data[1][0],matrix_A[1],NET,device,3)
heatmap_for_test(grad_torch.reshape(65,65),grad_np.reshape(65,65),label_list=['','',''],loss = True)

prime
prime
prime
prime
prime
prime
prime
prime
3 s ± 101 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
prime
prime
prime
prime
prime
prime
prime
prime
204 ms ± 3.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


NameError: name 'grad_torch' is not defined